In [1]:
#Mounting on the device
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
folder_path = '/content/drive/MyDrive/FML_Project'
print(folder_path)

/content/drive/MyDrive/FML_Project


In [3]:
import os
if os.path.isdir(folder_path):
    print("Folder exists")
else:
    print("Folder doesn't exist")

# List all files in the folder
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    print(file_path)

Folder exists
/content/drive/MyDrive/FML_Project/df_prime_test.csv
/content/drive/MyDrive/FML_Project/df_prime_train.csv
/content/drive/MyDrive/FML_Project/Prime_FULL
/content/drive/MyDrive/FML_Project/resnet50_features.pkl
/content/drive/MyDrive/FML_Project/resnet50_features_testfix.pkl
/content/drive/MyDrive/FML_Project/resnet50_features_final.pkl


In [4]:
import torch
from torch.utils.data import Dataset,DataLoader
from tqdm import tqdm
from torchvision import transforms
import numpy as np
import pandas as pd
from PIL import Image
import argparse
import os
import copy
import torch
import cv2

In [5]:
LABELS_Severity = {35: 0,
                   43: 0,
                   47: 1,
                   53: 1,
                   61: 2,
                   65: 2,
                   71: 2,
                   85: 2}


mean = (.1706)
std = (.2112)
normalize = transforms.Normalize(mean=mean, std=std)

transform = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    lambda x : x.expand(3,*x.shape[1:]),
    normalize,
])

In [6]:
class OCTDataset(Dataset):
    def __init__(self, args, subset='train', transform=None,):
        if subset == 'train':
            self.annot = pd.read_csv(args.annot_train_prime)
        elif subset == 'test':
            self.annot = pd.read_csv(args.annot_test_prime)
            
        self.annot['Severity_Label'] = [LABELS_Severity[drss] for drss in copy.deepcopy(self.annot['DRSS'].values)] 
        # print(self.annot)
        self.root = os.path.expanduser(args.data_root)
        self.transform = transform
        # self.subset = subset
        self.nb_classes=len(np.unique(list(LABELS_Severity.values())))
        self.path_list = self.annot['File_Path'].values
        self._labels = self.annot['Severity_Label'].values
        assert len(self.path_list) == len(self._labels)
        # idx_each_class = [[] for i in range(self.nb_classes)]

    def __getitem__(self, index):
        img, target = Image.open(self.root+self.path_list[index]).convert("L"), self._labels[index]

        if self.transform is not None:
            img = self.transform(img)

        return img, target

    def __len__(self):
        return len(self._labels)         

In [7]:
class NotebookArgs:
    def __init__(self, annot_test_prime = "/content/drive/MyDrive/FML_Project/df_prime_test.csv" , annot_train_prime = "/content/drive/MyDrive/FML_Project/df_prime_train.csv", data_root = "/content/drive/MyDrive/FML_Project"):
        self.annot_train_prime = annot_train_prime
        self.annot_test_prime = annot_test_prime
        self.data_root = data_root
args = NotebookArgs()

In [8]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.models import ResNet18_Weights

# Define the ResNet50 model
device = torch.device('cuda:0')
model = models.resnet18(weights=ResNet18_Weights.DEFAULT)

# Freeze all layers except the last 
for param in model.parameters():
    param.requires_grad = False
for param in model.layer4.parameters():
    param.requires_grad = True

# Replace the last fully connected layer with a new one that has 3 output features
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)

model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 190MB/s]


In [9]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Load the dataset and dataloaders
train_dataset = OCTDataset(args, 'train', transform=transform)
test_dataset = OCTDataset(args, 'test', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    # Training loop
    model.train()
    for images, labels in tqdm(train_loader):
        optimizer.zero_grad()
        images = images.to(device)
        labels= labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
   
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}")

100%|██████████| 758/758 [2:25:11<00:00, 11.49s/it]


Epoch 1, Loss: 0.5607


100%|██████████| 758/758 [02:46<00:00,  4.56it/s]


Epoch 2, Loss: 0.6961


100%|██████████| 758/758 [02:40<00:00,  4.72it/s]


...

Epoch 8, Loss: 0.0244


100%|██████████| 758/758 [02:39<00:00,  4.74it/s]


Epoch 9, Loss: 0.0317


100%|██████████| 758/758 [02:39<00:00,  4.74it/s]

Epoch 10, Loss: 0.0430


In [10]:
# Test the model
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total

print(f"Test Acc: {accuracy:.2f}%")

Test Acc: 43.41%
